In [2]:
%matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
# import gmplot
# from geopy.geocoders import Nominatim
# import time
# import gmaps
# import gmaps.datasets

Aijobsindustry_dataset = pd.read_csv("AIJobsIndustry_Clean_v3.csv")
print(f"No of rows in Aijobsindustry_dataset is {Aijobsindustry_dataset.shape[0]}")

Dice_us_jobs_dataset = pd.read_csv("Dice_US_jobs_Clean_v2.csv", encoding="cp437")
print(f"No of rows in Dice_us_jobs_dataset is {Dice_us_jobs_dataset.shape[0]}")

monster_dataset = pd.read_csv("Monster_Clean.csv")
print(f"No of rows in monster_dataset is {monster_dataset.shape[0]}")

indeed_dataset = pd.read_csv("alldata_Clean_v2.csv")
print(f"No of rows in indeed_dataset is {indeed_dataset.shape[0]}")

##file needs to match format for concat
# Dice_use_Tech_filtered = pd.read_csv("Dice_US_Tech_FilteredJobData.csv")
# Dice_use_Tech_filtered

print(f"No of rows in final merged data set should be is {Aijobsindustry_dataset.shape[0]+Dice_us_jobs_dataset.shape[0]+monster_dataset.shape[0]+indeed_dataset.shape[0]}")

datasets_to_merge = [Aijobsindustry_dataset,Dice_us_jobs_dataset,monster_dataset,indeed_dataset]

##please confirm the concat makes sense
merged_dataset = pd.concat(datasets_to_merge)
print(f"Actual no of rows in merged_dataset is {merged_dataset.shape[0]}")

Using matplotlib backend: Qt5Agg
No of rows in Aijobsindustry_dataset is 631
No of rows in Dice_us_jobs_dataset is 1282
No of rows in monster_dataset is 162
No of rows in indeed_dataset is 6953
No of rows in final merged data set should be is 9028
Actual no of rows in merged_dataset is 9028


In [3]:
def parser(column):
    word_count = {}
    for entry in column:
        entry = re.sub('[^A-Za-z0-9 ]+', '', str(entry))
#        print(entry)
        try:
            entry = entry.replace(',','')
            word_split = entry.split()
            for word in word_split:
                if word not in word_count:
                    word_count[word] = 1
                else:
                    word_count[word] += 1
        except AttributeError:
            print("Skipping an entry due to bad data")
            continue
    print(word_count)

In [5]:
desc = merged_dataset['Description']
parser(desc)

{'nan': 631, 'Informatica': 314, 'ETL': 1322, 'DeveloperSt': 1, 'Petersburg': 5, 'FL': 35, 'Only': 67, 'Full': 231, 'Time': 254, 'Direct': 132, 'HiresMultiple': 1, 'PositionsRequirements5': 1, 'years': 10549, 'experience': 21083, 'as': 23073, 'a': 67795, 'DI': 11, 'DeveloperInformatica': 1, 'Power': 218, 'Center': 1186, 'or': 36543, 'similar': 892, 'ETLDI': 1, 'toolInformatica': 1, 'B2B': 113, 'Data': 7881, 'Exchange': 40, 'SOA': 46, 'toolUnixLinux': 1, 'Operating': 116, 'System': 541, 'and': 239345, 'scripting': 723, 'languageDatabases': 1, 'Netezza': 25, 'Oracle': 635, 'MySQL': 424, 'SQLData': 4, 'Warehousing': 96, 'experienceStrong': 149, 'analytical': 3661, 'skills': 9420, 'presentation': 1004, 'skillDesiredExperience': 1, 'with': 61805, 'Database': 514, 'Administration': 288, 'working': 7451, 'frontend': 238, 'tools': 4955, 'Reporting': 315, 'etcC': 1, 'Java': 1581, 'C': 1348, 'programming': 2742, 'languageGood': 1, 'understanding': 4143, 'of': 101373, 'Application': 270, 'Interfa

In [ ]:
group_by_state = monster_dataset.groupby("Loc_state")
count_for_state = group_by_state.count()
count_for_state = count_for_state["Position"]
count_for_state